# Predicting Employee Attrition in the Dawn of Recession(Kaggle Competition)

## Summer Analytics 2020 Capstone Project

This was an Inclass Competition held by the Consulting and Analytics Club , IIT Guwahati as the final assignment
of their 6 week long Summer Analytics course. This was my first Kaggle Competition.

## Overview of Problem

As the COVID-19 keeps unleashing its havoc, the world continues to get pushed into the crisis of the great economic recession,
more and more companies start to cut down their underperforming employees. Companies firing hundreds and thousands of Employees
is a typical headline today. Cutting down employees or reducing an employee salary is a tough decision to take. It needs to be
taken with utmost care as imprecision in the identification of employees whose performance is attriting may lead to sabotaging
of both employees' career and the company's reputation in the market.

In [ ]:
#Importing nescessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split 
from sklearn import metrics
import xgboost as xgb
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


Now, let's import the data to the notebook i.e. train.csv


In [ ]:
df=pd.read_csv(r"./train.csv",index_col=0)
#now see what data looks like 
df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,CommunicationSkill,Behaviour
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,30,0,Non-Travel,Research & Development,2,3,Medical,571,3,Female,3,Laboratory Technician,4,Single,2564,0,No,14,3,0,12,2,11,7,6,7,4,1
2,36,0,Travel_Rarely,Research & Development,12,4,Life Sciences,1614,3,Female,3,Manufacturing Director,3,Married,4663,9,Yes,12,3,2,7,2,3,2,1,1,2,1
3,55,1,Travel_Rarely,Sales,2,1,Medical,842,3,Male,3,Sales Executive,4,Single,5160,4,No,16,3,0,12,3,9,7,7,3,5,1
4,39,0,Travel_Rarely,Research & Development,24,1,Life Sciences,2014,1,Male,3,Research Scientist,4,Single,4108,7,No,13,3,0,18,2,7,7,1,7,4,1
5,37,0,Travel_Rarely,Research & Development,3,3,Other,689,3,Male,3,Manufacturing Director,3,Married,9434,1,No,15,3,1,10,2,10,7,7,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1624,42,1,Travel_Frequently,Research & Development,19,3,Medical,752,3,Male,4,Research Scientist,3,Divorced,2759,6,Yes,12,3,0,7,2,2,2,2,2,3,1
1625,55,1,Travel_Rarely,Sales,2,1,Medical,842,3,Male,3,Sales Executive,4,Single,5160,4,No,16,3,0,12,3,9,7,7,3,5,1
1626,25,1,Travel_Rarely,Sales,9,2,Life Sciences,1439,1,Male,2,Sales Representative,1,Married,4400,3,No,12,3,0,6,2,3,2,2,2,5,1


Change all the categorical data into numerical data

In [ ]:
df.BusinessTravel[df.BusinessTravel == 'Non-Travel'] = 0
df.BusinessTravel[df.BusinessTravel == 'Travel_Rarely'] = 1
df.BusinessTravel[df.BusinessTravel == 'Travel_Frequently'] = 2
df.Department[df.Department == 'Research & Development'] = 0
df.Department[df.Department == 'Sales'] = 1
df.Department[df.Department == 'Human Resources'] = 2
df.EducationField[df.EducationField == 'Medical'] = 0
df.EducationField[df.EducationField == 'Life Sciences'] = 1
df.EducationField[df.EducationField == 'Other'] = 2
df.EducationField[df.EducationField == 'Marketing'] = 3
df.EducationField[df.EducationField == 'Technical Degree'] = 4
df.EducationField[df.EducationField == 'Human Resources'] = 5
df.MaritalStatus[df.MaritalStatus == 'Single'] = 0
df.MaritalStatus[df.MaritalStatus == 'Married'] = 1
df.MaritalStatus[df.MaritalStatus == 'Divorced'] = 2
df.Gender[df.Gender == 'Male'] = 0
df.Gender[df.Gender == 'Female'] = 1
df.OverTime[df.OverTime == 'No'] = 0
df.OverTime[df.OverTime == 'Yes'] = 1



In [ ]:
#lets check the no of rows and column of data and drop JobRole column
df = df.drop(['JobRole'], axis=1)
df.shape

(1628, 27)

Before starting the project lets look if the data has multiple or duplicate entries it is very crucial as it may affect the accuracy of model , it may show very high accuracy of model in training dataset but may not show in actual dataset because of duplicate or multiple entries.

In [ ]:
#these code will check for all duplicate or multiple entries and remove them
df.drop_duplicates(subset='EmployeeNumber', inplace=True)


In [ ]:
#new data has following number of rows and column
df.shape

(1000, 27)

In [ ]:
df

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,CommunicationSkill,Behaviour
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,30,0,0,0,2,3,0,571,3,1,3,4,0,2564,0,0,14,3,0,12,2,11,7,6,7,4,1
2,36,0,1,0,12,4,1,1614,3,1,3,3,1,4663,9,1,12,3,2,7,2,3,2,1,1,2,1
3,55,1,1,1,2,1,0,842,3,0,3,4,0,5160,4,0,16,3,0,12,3,9,7,7,3,5,1
4,39,0,1,0,24,1,1,2014,1,0,3,4,0,4108,7,0,13,3,0,18,2,7,7,1,7,4,1
5,37,0,1,0,3,3,2,689,3,0,3,3,1,9434,1,0,15,3,1,10,2,10,7,7,8,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,36,0,0,1,10,4,0,592,2,0,3,4,0,9980,1,0,14,3,0,10,3,10,3,9,7,4,1
997,40,0,1,0,16,3,1,1641,3,1,3,4,0,7945,6,1,15,3,0,18,2,4,2,3,3,2,1
998,46,1,1,1,9,2,0,118,3,0,2,4,0,9619,1,0,16,3,0,9,3,9,8,4,7,4,1


In [ ]:
y = df['Attrition']
X = df.drop('Attrition', axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=5000)

# Rough Models
We will try different models and see there accuracy to select which model will give us best result

## RandomForestClassifier

In [ ]:
xg = RandomForestClassifier()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.86


## DecisionTreeClassifier

In [ ]:
xg = DecisionTreeClassifier()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.79


## AdaBoostClassifier

In [ ]:
xg = AdaBoostClassifier()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.88


## LogisticRegression

In [ ]:
xg = LogisticRegression()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.84


## MLPClassifier

In [ ]:
xg = MLPClassifier()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.86


## SVC

In [ ]:
xg = SVC()
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.86


Among all the models AdaBoostClassifier gives us the best result , lets modify hyperparameters to get more accurate result

In [ ]:

xg = AdaBoostClassifier(base_estimator=None, n_estimators=50, learning_rate=.5, algorithm='SAMME', random_state=10)
xg.fit(X_train, y_train)
y_pred_class = xg.predict(X_test)



print(metrics.accuracy_score(y_test, y_pred_class))

0.9


Lets load the data to test data and predict the probabililty

In [ ]:
dft = pd.read_csv(r"./test.csv",index_col=0)
dft.BusinessTravel[dft.BusinessTravel == 'Non-Travel'] = 0
dft.BusinessTravel[dft.BusinessTravel == 'Travel_Rarely'] = 1
dft.BusinessTravel[dft.BusinessTravel == 'Travel_Frequently'] = 2
dft.Department[dft.Department == 'Research & Development'] = 0
dft.Department[dft.Department == 'Sales'] = 1
dft.Department[dft.Department == 'Human Resources'] = 2
dft.EducationField[dft.EducationField == 'Medical'] = 0
dft.EducationField[dft.EducationField == 'Life Sciences'] = 1
dft.EducationField[dft.EducationField == 'Other'] = 2
dft.EducationField[dft.EducationField == 'Marketing'] = 3
dft.EducationField[dft.EducationField == 'Technical Degree'] = 4
dft.EducationField[dft.EducationField == 'Human Resources'] = 5
dft.MaritalStatus[dft.MaritalStatus == 'Single'] = 0
dft.MaritalStatus[dft.MaritalStatus == 'Married'] = 1
dft.MaritalStatus[dft.MaritalStatus == 'Divorced'] = 2
dft.Gender[dft.Gender == 'Male'] = 0
dft.Gender[dft.Gender == 'Female'] = 1
dft.OverTime[dft.OverTime == 'No'] = 0
dft.OverTime[dft.OverTime == 'Yes'] = 1
dft = dft.drop(['JobRole'], axis=1)

In [ ]:
dft['Attrition'] = xg.predict_proba(dft)[::,1]


In [ ]:
dft['Attrition']

Id
1      0.395601
2      0.390713
3      0.424201
4      0.416335
5      0.372016
         ...   
466    0.455792
467    0.540774
468    0.434743
469    0.385691
470    0.362513
Name: Attrition, Length: 470, dtype: float64

In [ ]:
dft['Attrition'].to_csv (r'./hackathon.csv', index = True, header=True)